In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
import sklearn
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, GridSearchCV , cross_val_predict
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [3]:
#functions

def change_colnames(cols,df):
    df.columns = cols
    return df

def conv_int(df,col_name):
    myl = []
    for i in range(len(df[col_name])):
        if df[col_name][i] == 'Low':
            myl.append(0)
        elif df[col_name][i] == 'Medium':
            myl.append(1)
        
        else:
            myl.append(2)
    df.drop(col_name,axis=1)
    df[col_name] = myl
    return df
            
def change_astype(df,columns,dtype):
    dicti = {}
    for i in range(len(columns)):
        dicti[columns[i]] = dtype[i]
    print(dicti)
    df = df.astype(dicti)
    return df

def convdots(df,column):
    myl = []
    for i in range(len(df[column])):
        if df[column][i] == '.':
            
            myl.append(1)
        else:
            myl.append(df[column][i])
    df[column] = pd.DataFrame(myl)
    return df

def fillingna(df,cols):
    for col in cols:
        df[col] = df[col].fillna(df[col].mean())
    return df

def fillingna2(df,cols):
    for col in cols:
        df[col] = df[col].fillna(df[col].mode()[0])
    return df  

def rfr_model(X, y):# Perform Grid-Search
    gsc = GridSearchCV(
        estimator=RandomForestRegressor(),
        param_grid={
            'max_depth': range(3,7),
            'n_estimators': (4,10,50),
        },
        cv=5, scoring='neg_mean_squared_error', verbose=0,                         n_jobs=-1)
    
    grid_result = gsc.fit(X, y)
    best_params = grid_result.best_params_
    
    return scores,predictions,best_params

    


def RF_impute(df,target,rest):
    list_of_missin = df[df[colname].isna()].index.tolist()
    len(list_of_missin)
    rf1 = df.loc[~df[colname].isin([np.nan])][rest] 
    train_x, test_x, train_y, test_y = train_test_split(rf1[rest], rf1[target],
                                                        train_size=0.8,random_state=1)
    
    scores,predictions,bp = rfr_model(train_x,train_y)
    regressor = RandomForestRegressor(n_estimators = bp['n_estimators'],max_depth=bp['max_depth'], random_state = 0)
    regressor.fit(train_x, train_y)
    y_pred = regressor.predict(test_x)
    plt.scatter(y_pred,test_y)
    rf1_p = df.loc[df[target].isin([np.nan])][rest] 
    pred1 = regressor.predict(rf1_p.values)
    
def XGB_impute(df,target,rest):
    list_of_missin = df[df[colname].isna()].index.tolist()
    len(list_of_missin)
    xgb1 = df.loc[~df[colname].isin([np.nan])][rest] 
    train_x, test_x, train_y, test_y = train_test_split(xgb1[rest], xgb1[target],
                                                        train_size=0.8,random_state=1)
    
    scores,predictions,bp = rfr_model(train_x,train_y)
    model = xgb.XGBClassifier()
    model.fit(train_x, train_y)
    y_pred = model.predict(test_x)
    plt.scatter(y_pred,test_y)
    rf1_p = df.loc[df[target].isin([np.nan])][rest] 
    pred1 = model.predict(rf1_p.values)

In [4]:
col_names = [ 'ID','FICO','Business Revenue','TSR','Last 6M Avg Util','Income','Last 6M Avg Remit','Last 6M Avg Bal','Internal Revolve','Lending AR% in last 6M',
'ExternalBalance',
'Months in Business',
'preferred_spend_line',
'total_cards',
'preferred_bal_line',
'preavgremit1_3_all',
'External_rev_rate',
'Supplementary Cards',
'isChargeCust',
'CMV',
'Assigned Line']

col_names_cont = ['FICO','Business Revenue','TSR','Last 6M Avg Util','Income','Last 6M Avg Remit',
                  'Last 6M Avg Bal','Internal Revolve','Lending AR% in last 6M','ExternalBalance','preferred_bal_line',
                 'Months in Business','preferred_spend_line','preavgremit1_3_all','CMV','External_rev_rate']
col_names_cat = ['Supplementary Cards','isChargeCust','Assigned Line','total_cards']

In [5]:
train_raw = pd.read_csv('development_dataset.csv')
train_1 = change_colnames(col_names,train_raw)
train_1 = convdots(train_1,'total_cards')
train_1 = conv_int(train_1,'Assigned Line')
train_1_new = change_astype(train_1,['Assigned Line','total_cards'],['int64','int64'])
train_1_mean = fillingna(train_1_new,col_names_cont)
train_1_mean = fillingna2(train_1_mean,col_names_cat)

{'Assigned Line': 'int64', 'total_cards': 'int64'}


In [6]:
#Removing TSR ,Internal Revolve , External_rev_rate - too many missing values
train_2 = train_1_mean.drop(['TSR','Internal Revolve','External_rev_rate','ID'], axis=1)

In [7]:
#Average amount paid towards card bills in the last 3 months
train_3 = train_2.drop('Last 6M Avg Remit',axis=1)

#The other one is avg for 6M - which one to drop

In [8]:
train_raw = pd.read_csv('MoreLikeGods_IITKGP_final.csv')